In [ ]:
# Bibliotecas necessárias
import geopandas as gpd
import pandas as pd
import glob
import os
from pathlib import Path
import shutil

In [ ]:
def find_shape_files(directory_path):
    list_shape_files = []
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith('.shp'):
                file_path = Path(root) / file
                list_shape_files.append(file_path)
    return list_shape_files

def find_infos(row, directory_path):
    my_url = row['url']

    url_path = Path(my_url)
    relative_path = url_path.relative_to(directory_path)

    parts = list(relative_path.parts)
    if len(parts) >= 3:
        sub_1, sub_2, nome_shape = parts[-3:]
    else:
        sub_1, sub_2, nome_shape = '', '', parts[-1]
    return (
        str(url_path),
        sub_1,
        sub_2,
        nome_shape
    )

# deprecated
# def find_file_type(name):
#     return name.split('_', maxsplit=2)[2].split('.')[0]

def find_file_type(name):
    parts = name.split('_', maxsplit=2)
    if len(parts) < 3:
        return "Unknown"
    return parts[2].split('.')[0]

def get_path_f(folder):
    states = {}
    for state_dir in os.listdir(folder):
        if os.path.isdir(os.path.join(folder, state_dir)):
            state_name = state_dir
            state_path = os.path.join(folder, state_dir)
            states[state_name] = state_path
    return states

In [ ]:
input_path = r"C:\Users\guilh\Downloads\lote_rs\RS"
states_root = get_path_f(input_path)
output_path = Path(input_path)

In [ ]:
for estado, caminho in states_root.items():
    print(f"Estado: {estado}, Caminho: {caminho}")

    # Encontre todos os arquivos .shp no diretório e suas subpastas
    list_shape_files = find_shape_files(caminho)

    # Resultados
    #for shape_file in list_shape_files:
        #print(shape_file)

    # Crie Dataframe
    df = pd.DataFrame(list_shape_files, columns=['url'])

    # Crie colunas
    df[
        ['path', 'sub_1', 'sub_2', 'nome_shape']
    ] = df.apply(
        lambda row: pd.Series(find_infos(row, caminho)),
        axis=1,
    )
    df['nome_tipo'] = df['nome_shape'].apply(find_file_type)

    #Set Path Windows para o input path
    caminhoW = Path(caminho)

    # APP
    list_files_app = list(caminhoW.rglob('*APP/*APP.shp'))
    list_files_app_uso = list(caminhoW.rglob('*APP/*APP_USO.shp'))

    # HIDROGRAFIA
    list_files_hidro_simples = list(
        caminhoW.rglob('*HIDROGRAFIA/*RIOS_SIMPLES.shp')
    )
    list_files_hidro_duplas = list(
        caminhoW.rglob('*HIDROGRAFIA/*RIOS_DUPLOS*.shp')
    )
    list_files_hidro_nascentes = list(
        caminhoW.rglob('*HIDROGRAFIA/*NASCENTE*.shp')
    )
    list_files_hidro_massa = list(
        caminhoW.rglob('*HIDROGRAFIA/*MASSA*_DAGUA*.shp')
    )

    # USO
    list_files_uso = list(caminhoW.rglob('*USO/*USO*.shp'))

    #Dicionario de listas de dados
    dict_lists = {
        'app': list_files_app,
        'app_uso': list_files_app_uso,
        'hidro_simples': list_files_hidro_simples,
        'hidro_duplas': list_files_hidro_duplas,
        'hidro_nascentes': list_files_hidro_nascentes,
        'hidro_massa': list_files_hidro_massa,
        'uso': list_files_uso
    }
    # Loop sobre os itens do dicionario de listas de dados
    for k, v in dict_lists.items():
         #print(k)

        # Etapa 2: Junta tudo!
         list_shp = [gpd.read_file(shp).to_crs(epsg=4326) for shp in v]
         gdf = gpd.GeoDataFrame(pd.concat(list_shp, ignore_index=True), crs=4326)

         # Etapa 3: Save
         gdf.to_file(
              output_path / f'{estado}_{k}.gpkg',
              layer=f'{k}',
              driver='GPKG',
              OVERWRITE=True
         )
         print(f'Fim do {k}')
        # Liberar variáveis após cada iteração
    del list_shape_files, df, caminhoW, list_files_app, list_files_app_uso
    del list_files_hidro_simples, list_files_hidro_duplas, list_files_hidro_nascentes, list_files_hidro_massa
    del list_files_uso, dict_lists, list_shp, gdf
    # End
    print('Fim da iteracao')